In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
%matplotlib inline

In [ ]:
df_red=pd.read_csv('winequality-red.csv',delimiter=';')

In [ ]:
df_white=pd.read_csv('winequality-white.csv',delimiter=';')

In [ ]:
df_red

In [ ]:
df_white

In [ ]:
profile_red = ProfileReport(df_red, title='Pandas Profiling Report', html={'style':{'full_width':True}})

In [ ]:
profile_white= ProfileReport(df_white, title='Pandas Profiling Report', html={'style':{'full_width':True}})

In [ ]:
profile_red

In [ ]:
profile_red.to_file(output_file="your_report.html")

In [ ]:
profile_white

In [ ]:
profile_white.to_file(output_file="your_report_white.html")

In [ ]:
# removing the duplicates rows in red wine's table
df_red.drop_duplicates(inplace=True)

In [ ]:
df_red.shape

In [ ]:
# removing the duplicates rows in white wine's table
df_white.drop_duplicates(inplace=True)

In [ ]:
df_white.shape

In [ ]:
# calculating the quality median for the two tables
print(df_red['quality'].median())
print(df_white['quality'].median())

In [ ]:
# change the label for red and white wine's tables
# red wine
df_red['target']=np.where(df_red['quality']<6,0,1)

# white wine
df_white['target']=np.where(df_white['quality']<6,0,1)

In [ ]:
df_red.head()

In [ ]:
# dropping quality column in red wine's table
df_red.drop('quality',axis=1,inplace=True)

In [ ]:
df_white.head()

In [ ]:
# dropping quality column in white wine's table
df_white.drop('quality',axis=1,inplace=True)

In [ ]:
# checking that the data is balanced
df_red['target'].value_counts()/len(df_red)

In [ ]:
# checking that the data is balanced
df_white['target'].value_counts()/len(df_white)

In [ ]:
df_white = pd.concat([df_white,df_white[df_white['target']==0]],axis=0)

In [ ]:
# checking that the data is balanced
df_white['target'].value_counts()/len(df_white)

In [ ]:
# defining the features and the target
# red
X_red = df_red.iloc[:,0:-1]
y_red = df_red.iloc[:,-1]

# wine
X_white = df_white.iloc[:,0:-1]
y_white = df_white.iloc[:,-1]

In [ ]:
print(X_white.shape,y_white.shape)

In [ ]:
df_white['target'].value_counts()

In [ ]:
df_white.shape

In [ ]:
# calculating the VIF for features selections

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# For each X, calculate VIF and save in dataframe

# red
vif_red = pd.DataFrame()
vif_red["VIF Factor"] = [variance_inflation_factor(X_red.values, i) for i in range(X_red.shape[1])]
vif_red["features"] = X_red.columns

In [ ]:
# white
vif_white = pd.DataFrame()
vif_white["VIF Factor"] = [variance_inflation_factor(X_white.values, i) for i in range(X_white.shape[1])]
vif_white["features"] = X_white.columns

In [ ]:
vif_red.to_csv('vif_red.csv')

In [ ]:
vif_white.to_csv('vif_white.csv')

In [ ]:
# choosing the features
vif_red

In [ ]:
vif_white

In [ ]:
# drop the features that we don't need
X_red.drop(['density','pH','alcohol','fixed acidity'],axis=1, inplace=True)

X_white.drop(['density','pH','alcohol','fixed acidity'],axis=1, inplace=True)

In [ ]:
X_red.shape

In [ ]:
y_red.shape

In [ ]:
# split the data into train and test
from sklearn.model_selection import train_test_split

X_red_train, X_red_test,y_red_train,y_red_test = train_test_split(X_red, y_red, test_size=0.20, random_state=42,stratify=y_red)

In [ ]:
X_white_train, X_white_test,y_white_train,y_white_test = train_test_split(X_white, y_white, test_size=0.20, random_state=42,stratify=y_white)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
estimators = [('scaling', MinMaxScaler()),
              ('model', LogisticRegression())]
pipe = Pipeline(estimators)
param_grid = [
{'scaling':[StandardScaler(),MinMaxScaler(), None], 
 'model': [LogisticRegression()],
 'model__C':np.power(10.0, np.arange(-5, 5))},
{'scaling':[StandardScaler(),MinMaxScaler(), None], 
 'model': [SVC()],
 'model__C':np.power(10.0, np.arange(-5, 5)),
 'model__kernel':['rbf','linear','sigmoid'],
 'model__gamma':[1, 10, 100]}
]
grid_red = GridSearchCV(pipe, param_grid, cv=5, verbose=1, n_jobs=-1)
fittedgrid_red = grid_red.fit(X_red_train, y_red_train)

In [ ]:
estimators = [('scaling', MinMaxScaler()),
              ('model', LogisticRegression())]
pipe = Pipeline(estimators)
param_grid = [
{'scaling':[StandardScaler(),MinMaxScaler(), None], 
 'model': [LogisticRegression()],
 'model__C':np.power(10.0, np.arange(-5, 5))},
{'scaling':[StandardScaler(),MinMaxScaler(), None], 
 'model': [SVC()],
 'model__C':np.power(10.0, np.arange(-5, 5)),
 'model__kernel':['rbf']}
]
grid_white = GridSearchCV(pipe, param_grid, cv=5, verbose=1, n_jobs=-1)
fittedgrid_white = grid_white.fit(X_white_train, y_white_train)

In [ ]:
fittedgrid_red.best_estimator_

In [ ]:
fittedgrid_red.score(X_red_test, y_red_test)

In [ ]:
fittedgrid_white.best_estimator_

In [ ]:
fittedgrid_white.score(X_white_test, y_white_test)